# Tensorflow Linear Regression Single Feature Part 2

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# yahoo finance is used to fetch data 
import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2007-01-01'
end = '2018-11-01'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2007-01-03,19.520000,19.520000,20.400000,19.350000,20.080000,28350300
2007-01-04,19.790001,19.790001,19.860001,19.320000,19.660000,23652500
2007-01-05,19.709999,19.709999,19.910000,19.540001,19.540001,15902400
2007-01-08,19.469999,19.469999,19.860001,19.370001,19.709999,15814800
2007-01-09,19.650000,19.650000,19.709999,19.370001,19.450001,14494200


In [3]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

,Adj Close,Close,High,Low,Open,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2007-01-04,19.790001,19.790001,19.860001,19.320000,19.660000,23652500,0,0,0,0.013832
2007-01-05,19.709999,19.709999,19.910000,19.540001,19.540001,15902400,0,1,0,-0.004043
2007-01-08,19.469999,19.469999,19.860001,19.370001,19.709999,15814800,0,0,1,-0.012177
2007-01-09,19.650000,19.650000,19.709999,19.370001,19.450001,14494200,1,1,1,0.009245
2007-01-10,20.010000,20.010000,20.020000,19.500000,19.639999,19783200,1,1,1,0.018321


In [4]:
dataset.shape

(2979, 10)

In [5]:
import tensorflow as tf

print(tf.__version__)

1.10.0


In [6]:
feature = np.array(dataset['Returns'])
target = np.array(dataset['Adj Close'])

In [7]:
from sklearn.model_selection import train_test_split  
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size = 1/4, random_state = 0)

In [8]:
# TF graph input
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

In [9]:
# Set model weights
w = tf.get_variable('weights', initializer=tf.constant(0.0))
b = tf.get_variable('bias', initializer=tf.constant(0.0))

In [10]:
# Set parameters
learning_rate = 0.1
training_iteration = 200
# Construct a linear model
model = X * w + b

In [11]:
samples_number = feature.size

In [12]:
# Minimize squared errors
cost_function = tf.reduce_sum(tf.pow(model - Y, 2))/(2 * samples_number) #L2 loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function) #Gradient descent

In [13]:
# Initialize variables
init = tf.global_variables_initializer()

In [14]:
sess=tf.Session()
sess.run(init)

In [15]:
sess.run(optimizer,{X:X_test,Y:Y_test})

In [16]:
sess.run([w,b])

[0.0004692331, 0.17247967]